# Exercise 1 - Signal processing

In this practical we will use a signal processing based workflow in phenopype to segment some butterflies of the genus Junonia. The images stem from museum collections and are highly standardized, therefore signal processing will work well for us.

<center><img src=../assets/butterfly-example.jpg width="500" ></center>


We start by importing phenopype (www.phenopype.org). Here we also set our project working 

In [ ]:
import phenopype as pp
import os
import cv2
import matplotlib.pyplot as plt

os.chdir("D:\git-repos\mluerig\phenomics-workshop-aussois") 

<div class="alert-warning">
NOTE: This assumes that you <a href="https://www.dropbox.com/scl/fo/ohva31yvrszje221ratpg/ANQdv85xdhkgkwctR5es5OA?rlkey=4kvcsjmlndms70otqsr8ui06y&dl=0" target="_blank">downloaded the data-archive</a> from dropbox and placed the "data_raw" folder in the github-repo.
</div>

## Python mode

Good for self-education, prototyping, custom workflows, ... 

In [ ]:
## load image
img = pp.load_image(r"data_raw\images_set1\junonia_coenia\a7d622d7-b4ff-476c-b61f-2229490aa713_1.jpg")

In [ ]:
## show image (close with enter)
pp.show_image(img)

In [ ]:
## create a mask
annotations = pp.preprocessing.create_mask(img)
annotations

In [ ]:
## select gray channel (many if not most SP algorithms need single channel)
img_gray = pp.preprocessing.decompose_image(img, channel="gray")

In [ ]:
## threshold algorithm to produce binary mask
img_mask = pp.segmentation.threshold(img_gray, method="binary", annotations=annotations)
pp.show_image(img_mask)

In [ ]:
## what did we do here:
hist = cv2.calcHist([img_gray], [0], None, [256], [0, 256])
plt.figure(figsize=(10, 5))
plt.plot(hist, color='black', linewidth=1.5)
plt.axvline(x=127, color='red', linewidth=2)

In [ ]:
## bad choice of cutoff value
img_mask = pp.segmentation.threshold(img_gray, method="binary", value=100, annotations=annotations)
pp.show_image(img_mask)

In [ ]:
## better choice for cutoff
img_mask = pp.segmentation.threshold(img_gray, method="binary", value=160, annotations=annotations)
pp.show_image(img_mask)

In [ ]:
img_blurred = pp.preprocessing.blur(img, kernel_size=5)
img_mask = pp.segmentation.threshold(img_blurred, method="binary", value=160, annotations=annotations)
pp.show_image(img_mask)

In [ ]:
hist = cv2.calcHist([img_gray], [0], None, [256], [0, 256])
plt.figure(figsize=(10, 5))
plt.plot(hist, color='black', linewidth=1.5)
plt.axvline(x=160, color='red', linewidth=2)

In [ ]:
## detect contour in binary image - spits out a json file
pp.segmentation.detect_contour(img_mask, annotations=annotations)

In [ ]:
## filter!
annotations = pp.segmentation.detect_contour(img_mask, annotations=annotations, min_area=1000)

In [ ]:
## let's measure shape traits
annotations = pp.measurement.compute_shape_features(annotations)

In [ ]:
## let's measure texture traits (pyradiomics)
annotations = pp.measurement.compute_texture_features(img, annotations)

In [ ]:
## create a canvas and draw annotations as image elements
canvas = pp.visualization.select_canvas(img)
canvas = pp.visualization.draw_mask(canvas, annotations, line_colour="blue")
canvas = pp.visualization.draw_contour(canvas, annotations)
pp.show_image(canvas)

In [ ]:
## export canvas, and annotations as json and csv (csv needs image name)
pp.export.save_canvas(canvas, dir_path="phenopype")
pp.export.save_annotation(annotations, file_name=r"annotations1.json", dir_path="phenopype")
pp.export.export_csv(annotations, image_name="a7d622d7-b4ff-476c-b61f-2229490aa713_1.jpg", dir_path="phenopype")

## Using the interactive workflow

Here, instead of writing down our analysis as a sequence of Python code, as we did in the low throughput workflow, we supply the same functions through a configuration file in human readable YAML format. This file can then be loaded by phenopype’s Pype class, which initiates the analysis by triggering three actions:

1. open the YAML configuration file in the default OS text editor
2. parse the contained functions and execute them in the sequence
3. open a HighGUI window showing the processed image, updates with every step

After an iteration of all steps, users can evaluate the results and decide to modify the opened configuration file (e.g. either change function parameters or add new functions), and run Pype again (by saving the changes), or to terminate the Pype-run and save all results to the root folder of the image (using Ctrl+Enter).


In [ ]:
pp.Pype(image_path=r"data_raw\images_set1\junonia_coenia\a7d622d7-b4ff-476c-b61f-2229490aa713_1.jpg", 
        tag="01", config_path="templates/01-thresholding.yaml", dir_path="phenopype")

## Using projects
Now that we know what happens behind the scences move on and make a phenopype project to work . If the downloaded git repo is my project main directory, then I would store different phenopype projects in a subfolder - let's name it "phenopype" and the project "project1":

In [ ]:
proj = pp.Project("phenopype/project1")

Next we import images to the project: first we use `images_set1`:

In [ ]:
proj.add_files("data_raw\images_set1",recursive=True)

Next we use `add_config`add configuration files to each project 

In [ ]:
proj.add_config(tag="01", template_path = "templates/01-thresholding.yaml")

Finally, we go through all directories using a simple for loop:

In [ ]:
for path in proj.dir_paths:
    p1 = pp.Pype(path, tag="01")

If we need to modify a config and adjust it for all, we can do so here by setting overwrite=True:

In [ ]:
proj.add_config(tag="01", template_path = "templates/01-thresholding.yaml", overwrite=True)

In [ ]:
for path in proj.dir_paths:
    p1 = pp.Pype(path, tag="01")

## Using different config files with "tag"

To add different config files to the same project use different tags:

In [ ]:
proj.add_config(tag="02", template_path = "templates/02-thresholding.yaml", overwrite=True)

In [ ]:
for path in proj.dir_paths:
    p1 = pp.Pype(path, tag="02")